In [1]:
from konlpy.tag import Mecab

import re
from collections import Counter
import pandas as pd

In [2]:
# 데이터 불러오기
data = pd.read_csv('./youtube_channel_comments_data_20240606_104600.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56453 entries, 0 to 56452
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   channelId           56453 non-null  object
 1   channelTitle        56453 non-null  object
 2   channelPublishedAt  56453 non-null  object
 3   subscriberCount     56453 non-null  int64 
 4   videoId             56453 non-null  object
 5   videoTitle          56453 non-null  object
 6   videoAuthorId       56453 non-null  object
 7   videoPublishedAt    56453 non-null  object
 8   duration            56453 non-null  object
 9   viewCount           56453 non-null  int64 
 10  likeCount           56453 non-null  int64 
 11  dislikeCount        56453 non-null  int64 
 12  commentId           56453 non-null  object
 13  commentAuthor       56325 non-null  object
 14  authorId            56453 non-null  object
 15  commentText         56449 non-null  object
 16  commentLikeCount    56

In [4]:
# cleaned_commentText의 데이터 보기
data['commentText']


0                         야채 많이드시고 가끔 요구르트 챙겨드시면\n어느정도 예방됨
1                            제일 확실한건 변비 설사 반복 됩니다 꼭 검사해보셔요
2                                           좋은정보 감사합니다 😊😊😊
3                                        걸리면 하느님 만나로 올라갈게요
4                                이중에 하나도 해당 안됐는데\n대장암 4기..
                               ...                        
56448            이지약사님 정말 오랜만에 뵙네요 그동안 잘 지내셨나요\n너무 보고 싶었어요
56449                                     감사합니다 잘 지내셨죠~~!!
56450    이지 약사님, 정말 오랜만에 뵙네요. 그동안 잘 지내셨나요?\n너무 보고 싶었어요. ^^
56451            안녕하세요 감사합니다..ㅎㅎ 부활해 보았습니당..ㅎㅎ 무더위 잘 보내셨죠!
56452                 @@ezyaksa 이지 약사님을 이렇게 뵈니 너무 반갑네요. ㅎㅎ
Name: commentText, Length: 56453, dtype: object

In [5]:
# 특수 문자 제거 함수
def clean_text(text):
    # 줄바꿈 문자 포함하여 특수 문자 제거
    text = re.sub(r'[^ㄱ-ㅎ가-힣0-9\s]', '', text)
    # "ㅎㅎ", "ㅋㅋ" 등 비정식 단어 제거
    text = re.sub(r'ㅎㅎ|ㅋㅋ', '', text)
    text = re.sub(r'\s+', ' ', text).strip()  # 공백 문자를 하나의 공백으로 치환하고 양쪽 공백 제거
    return text

In [6]:
# NaN 값 제거
data = data.dropna(subset=['commentText'])

# NaN 값 공백 문자열로 대체
data['commentText'] = data['commentText'].fillna('')

# 댓글, 답글 데이터 전처리
data['cleaned_commentText'] = data['commentText'].apply(clean_text)

In [7]:
data['cleaned_commentText']

0                  야채 많이드시고 가끔 요구르트 챙겨드시면 어느정도 예방됨
1                    제일 확실한건 변비 설사 반복 됩니다 꼭 검사해보셔요
2                                       좋은정보 감사합니다
3                                걸리면 하느님 만나로 올라갈게요
4                           이중에 하나도 해당 안됐는데 대장암 4기
                           ...                    
56448     이지약사님 정말 오랜만에 뵙네요 그동안 잘 지내셨나요 너무 보고 싶었어요
56449                                 감사합니다 잘 지내셨죠
56450    이지 약사님 정말 오랜만에 뵙네요 그동안 잘 지내셨나요 너무 보고 싶었어요
56451             안녕하세요 감사합니다 부활해 보았습니당 무더위 잘 보내셨죠
56452                       이지 약사님을 이렇게 뵈니 너무 반갑네요
Name: cleaned_commentText, Length: 56449, dtype: object

In [8]:
m = Mecab()

print(m.pos("순대국 먹고 싶다."))
print(m.pos("순댓국 먹고 싶다."))
print(m.pos("패스트파이브에서 일을 합니다."))
print(m.pos("아이오아이는 정말 이뻐요."))

[('순', 'MM'), ('대국', 'NNG'), ('먹', 'VV'), ('고', 'EC'), ('싶', 'VX'), ('다', 'EF'), ('.', 'SF')]
[('순댓국', 'NNG'), ('먹', 'VV'), ('고', 'EC'), ('싶', 'VX'), ('다', 'EF'), ('.', 'SF')]
[('패스트', 'NNP'), ('파이브', 'NNP'), ('에서', 'JKB'), ('일', 'NNG'), ('을', 'JKO'), ('합니다', 'VV+EF'), ('.', 'SF')]
[('아이오', 'NNG'), ('아이', 'NNG'), ('는', 'JX'), ('정말', 'MAG'), ('이뻐요', 'VA+EF'), ('.', 'SF')]


In [9]:
def extract_nouns(text):
    """
    주어진 텍스트에서 명사만 추출하여 반환하는 함수.
    
    Args:
    text (str): 명사를 추출할 텍스트.
    
    Returns:
    list: 추출된 명사들의 리스트.
    """
    # Mecab의 noun 메소드를 사용하여 명사 추출
    nouns = m.nouns(text)
    return nouns

In [10]:
# 예제 사용
example_text = "패스트파이브에서 일을 합니다. 아이오아이는 정말 이뻐요."
nouns = extract_nouns(example_text)
print(nouns)

['패스트', '파이브', '일', '오', '아이']


### 2. 텍스트 토크나이징
토크나이저를 사용하여 텍스트 데이터를 토큰화합니다.



In [11]:

data['tokenized'] = data['cleaned_commentText'].apply(extract_nouns)


In [12]:
# LTokenizer
# 한글자, 지시대명사 등 제거
data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if len(item) > 1])

# 한글자 이외의 형용사, 부사, 동사 불용어 stopword 목록 작성하여 반영
stopword = ['ㅎㅎ', 'ㅋㅋ', 'ㅠㅠ', 'ㅜㅜ', 'ㅇㅇ', 'ㅈㅈ', 'ㅊㅊ', 'ㅋㅋ', 'ㅎㅎ', 'ㅇㅇ', 'ㅈㅈ', 'ㅊㅊ']
data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopword])


In [13]:
# 각 행에 대해 l_tokenized와 maxscore_tokenized 결과를 함께 출력
for index, row in data.iterrows():
    print(f"Row {index}:")
    print("Tokenized: ", row['tokenized'])
    # print("MaxScore Tokenized: ", row['maxscore_tokenized'])
    # print("\n")
    if index > 100:
        break


Row 0:
Tokenized:  ['야채', '요구르트', '정도', '예방']
Row 1:
Tokenized:  ['변비', '반복', '검사']
Row 2:
Tokenized:  ['정보', '감사']
Row 3:
Tokenized:  ['하느님', '만나', '게요']
Row 4:
Tokenized:  ['하나', '해당', '대장암']
Row 5:
Tokenized:  []
Row 6:
Tokenized:  []
Row 7:
Tokenized:  []
Row 8:
Tokenized:  []
Row 9:
Tokenized:  ['체중', '감소', '안심']
Row 10:
Tokenized:  ['질병', '사람']
Row 11:
Tokenized:  ['내츄럴', '터스', '베르베린', '휴지기']
Row 12:
Tokenized:  ['마지막', '제품', '내츄럴', '터스', '베르베린', '리포', '미셀', '구매', '하루', '섭취', '건가요']
Row 13:
Tokenized:  ['당뇨', '환자', '영양제']
Row 14:
Tokenized:  ['요즘', '알코', '클렌', '니코틴', '알코올', '분해', '영양제', '광고', '약국', '판매', '효과', '건가요']
Row 15:
Tokenized:  ['당뇨병', '당뇨병', '채소', '결국', '당뇨', '경우', '영양제', '지병', '영양제', '유산균', '비타민', '비타민', '마그네슘', '엽산', '가지', '간수', '검사', '당뇨']
Row 16:
Tokenized:  ['오메가', '부작용']
Row 17:
Tokenized:  ['바나바', '베르베린', '섭취', '당뇨', '환자', '개월', '정도', '증상']
Row 18:
Tokenized:  ['약사', '평소', '폐경', '이후', '영양제', '에스트로겐', '섭취', '얘기']
Row 19:
Tokenized:  ['폐경', '이후', '섭취', '기본', '마비', 

In [14]:
# 채널별로 데이터 그룹화 및 토큰 리스트 합치기
grouped_data = data.groupby('channelId')['tokenized'].agg(lambda x: [token for sublist in x for token in sublist])
grouped_data


channelId
UC3iSLVH0MxHfwO69oHKpvog    [리틀, 약사, 최고, 감사, 비문증, 브로멜, 라인, 파파인, 설명, 리틀, 약사...
UC6ggXTuBVchhwHeQ12Ita1w    [베르베린, 복용, 방법, 안내, 복용량, 점진, 증량, 처음, 하루, 시작, 하루...
UCCMFTDGarjgZLc1DlIbbvRg    [감사, 사람, 소화, 열량, 셀룰로오스, 흡수, 스펀지, 역할, 약사, 스타, 스...
UCMFk5S7g5DY-CZNVh_Kyz_A    [야채, 요구르트, 정도, 예방, 변비, 반복, 검사, 정보, 감사, 하느님, 만나...
UCY-mXLM6DsS9cmSwlh0tqSA    [하루, 아르기닌, 변비, 유산균, 공구, 오픈, 구매, 링크, 트랜, 유산균, 유...
Name: tokenized, dtype: object

In [15]:
# 각 채널별로 상위 20개 단어 추출
top_words_per_channel = {}
for channel_id, tokens in grouped_data.items():
    # 토큰의 빈도수 계산
    token_counts = Counter(tokens)
    # 가장 빈번한 30개 단어 추출
    top_words = token_counts.most_common(30)
    top_words_per_channel[channel_id] = top_words

In [16]:
# 결과 출력
for channel_id, words in top_words_per_channel.items():
    print(f"Channel {channel_id}:")
    for word, count in words:
        print(f"{word}: {count}")
    print("\n")

Channel UC3iSLVH0MxHfwO69oHKpvog:
감사: 1333
약사: 1049
비타민: 745
마그네슘: 640
상담: 615
제품: 592
영양제: 580
리틀: 562
영상: 522
밴드: 517
복용: 516
오메가: 494
사람: 410
건강: 401
네이버: 399
섭취: 376
정도: 369
효과: 361
도움: 352
정보: 337
하루: 324
답변: 309
커피: 303
유튜브: 286
시간: 285
생각: 282
음식: 277
문제: 252
이용: 249
불편: 245


Channel UC6ggXTuBVchhwHeQ12Ita1w:
감사: 712
복용: 658
약사: 590
영상: 335
제품: 270
섭취: 263
영양제: 244
정보: 235
효과: 233
레스: 198
선생: 178
건강: 177
피세: 173
베르베린: 166
멜라토닌: 165
도움: 164
안녕: 144
추천: 142
미국: 139
비타민: 129
아침: 123
구매: 122
베라트: 119
정도: 118
하루: 113
베라: 113
피부: 98
구입: 94
개월: 87
트롤: 87


Channel UCCMFTDGarjgZLc1DlIbbvRg:
감사: 90
크림: 46
약사: 45
효과: 37
지약: 31
제품: 28
멜라토: 25
정보: 22
피부: 21
영상: 20
노사: 20
멜라: 19
부위: 16
여드름: 15
색소: 15
이지: 14
약국: 14
오랜만: 14
침착: 13
흉터: 13
흡수: 12
안녕: 12
자외선: 12
처방: 11
얼굴: 11
추천: 11
이거: 10
함량: 10
정도: 10
소화: 9


Channel UCMFk5S7g5DY-CZNVh_Kyz_A:
감사: 8574
섭취: 5627
약사: 4468
비타민: 4299
영양제: 3742
오메가: 3700
제품: 3412
복용: 3104
영상: 2859
도움: 2488
시청: 2363
추천: 2279
마그네슘: 1897
부탁: 1876
생각: 1819
말씀: 1662
정도: 